In [5]:
import pandas as pd
import numpy as np 
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
warnings.filterwarnings('ignore')
from collections import deque
import joblib
import datetime



class IPEpricePrediction:
    ''' A simple class to easily query the ETS model from inside AnyLogic using Pypeline '''
    def __init__(self):
        self.ipe_master_ = pd.read_csv('IPE Master_Update.csv')
        
        self.modelSP = joblib.load("RP.pkl")
        self.modelMOP = joblib.load("MOP.pkl")
    
    def predict_priceSP(self, reseller_code, sku_code, qty_):
        ipe_master = self.ipe_master_
        master_data = ipe_master[(ipe_master.Reseller==reseller_code) & (ipe_master.SKU==sku_code)]
        if master_data.shape[0]==0:
            master_data = ipe_master.iloc[0, :]
        master_data['Final_Qty_var'] = qty_
        prediction =  self.modelSP.predict(master_data[['FINAL_GROSS_MARGIN', 'Final_Qty_var','Sub_Market_Segment_val',
                             'discount_last_10_transactions','sup_lagged_median']])
        return round(prediction[0],2)
    

    def predict_priceMOP(self, reseller_code, sku_code, qty_, SP):
        ipe_master = self.ipe_master_
        master_data = ipe_master[(ipe_master.Reseller==reseller_code) & (ipe_master.SKU==sku_code)].reset_index(drop=True)
        if master_data.shape[0]==0:
            master_data = ipe_master.iloc[0, :]
        master_data['Final_Qty_var'] = qty_
        master_data['Approved Net Price'] = SP
        
        input_= master_data[['Video_seg', 'sup_lagged_median', 'itd_net_pos_revenue_Flag',
       'Quote_Recency_Flag', 'Plan_TAM_ratio_Flag',
       'Share_Change_Overall_Flag', 'Weeks_on_hand_inventory',
       'Approved Net Price']]
        # scalling 
        mop_features = input_.columns


        ## if some MOP input variable is missing:
        for i in mop_features:
            if i not in master_data.columns:
                print('Warning: Variable ',i,' missing in FE Scoring script output for MOP')

                print(i)
                master_data[i]=0
            else:
                continue
        mop_scoring=input_[mop_features]
        for i in mop_scoring.columns:
            mop_scoring[i]=pd.to_numeric(mop_scoring[i])



        mop_scoring_scaled=pd.DataFrame()
        for col in mop_features:
            try:

                scaler_pickle=joblib.load(col+'.pkl')
                #print(f'loaded {col} pickle')
                mop_scoring_scaled_1=pd.DataFrame(scaler_pickle.transform(pd.DataFrame(mop_scoring[col])))
                mop_scoring_scaled_1.columns=[col]
                mop_scoring_scaled=pd.concat([mop_scoring_scaled,mop_scoring_scaled_1],axis=1)
                del mop_scoring_scaled_1
            except:
                continue
        #         print(col,': No pickle for binary variables')

        # adding the scaled features column to the dataset
        non_scaled_features=[s for s in mop_features if s not in mop_scoring_scaled.columns]
        for col in non_scaled_features:
            mop_scoring_scaled[col]=mop_scoring[col]

        del mop_scoring
        mop_scoring=mop_scoring_scaled.copy()
        mop_scoring.columns=mop_scoring.columns+'_scaled'
        score_df_90 = pd.DataFrame()
        score_df_90=pd.concat([score_df_90,mop_scoring,master_data],axis=1)
        del mop_scoring




        mop_scoring_scaled=mop_scoring_scaled.replace(np.nan,0,regex=True)
        mop_scoring_scaled= mop_scoring_scaled[['Video_seg', 'sup_lagged_median', 'itd_net_pos_revenue_Flag',
       'Quote_Recency_Flag', 'Plan_TAM_ratio_Flag',
       'Share_Change_Overall_Flag', 'Weeks_on_hand_inventory',
       'Approved Net Price']]
        
        
        #logreg=joblib.load(os.getcwd()+PICKLE+MOP_PICKLE)
        logreg = joblib.load("MOP.pkl")

        if len(mop_scoring_scaled)>0:
#             print(mop_scoring_scaled.columns)
            pred_mop=logreg.predict_proba(mop_scoring_scaled)
            print(pred_mop[:,1])

            #calculating win probability
            # listing features and coeff
            listlogregcoef=logreg.coef_.tolist()
            logregeq=pd.DataFrame(zip(mop_features,listlogregcoef[0]),columns=["features","coefficients"])
            coeff_trans = logregeq.set_index('features').T
            coeff_trans_1=coeff_trans.drop(['Approved Net Price'], axis=1)
            mop_scoring_scaled=mop_scoring_scaled[mop_features]



            # calculate the win prob

            score_df_90['Logr_Predictions']=pred_mop[:,1]
            score_df_90['Logistic_Regression_Score'] = mop_scoring_scaled.multiply(np.array(coeff_trans), axis='columns').sum(axis=1) +logreg.intercept_[0]#pred_mop[:,1]
            score_df_90['Win_Probability'] = np.exp(score_df_90['Logistic_Regression_Score'])/(1+ np.exp(score_df_90['Logistic_Regression_Score']))
            print(score_df_90['Win_Probability'])
            #             self.ipe_master_['prob'] = score_df_90['Win_Probability']

            # calculating the impact of all variables except ANP
            mop_scoring_scaled_1=mop_scoring_scaled.drop(['Approved Net Price'],axis=1)
            score_df_90['Subtract_Logistic_Regression_Score'] = mop_scoring_scaled_1.multiply(np.array(coeff_trans_1), axis='columns').sum(axis=1) +logreg.intercept_[0]

            score_df_90['Target_winrate_Cap'] = score_df_90['Win_Probability'] * 0.995       
            score_df_90['Target_winrate'] = np.where (((score_df_90['Win_Probability']>  score_df_90['Target_winrate']) & (score_df_90['Target_winrate']>score_df_90['Target_winrate_Cap'])),
                                                          score_df_90['Target_winrate'], 
                                                           np.where((score_df_90['Win_Probability']<  score_df_90['Target_winrate']),
                                                                      score_df_90['Win_Probability'],score_df_90['Target_winrate_Cap']))

            # back calculating the Price - scaled right now
            score_df_90['Win_Threshold_LRScore'] = np.log(score_df_90['Target_winrate']/(1- score_df_90['Target_winrate']))
            score_df_90['Win_Threshold_Scaled_Price'] = (score_df_90['Win_Threshold_LRScore'] - score_df_90['Subtract_Logistic_Regression_Score'])/coeff_trans ['Approved Net Price'][0]

            # print(score_df_90['Win_Threshold_Scaled_Price'].min(),score_df_90['Win_Threshold_Scaled_Price'].max())

            score_df_90['Win_Threshold_Scaled_Price']=np.where(score_df_90['Win_Threshold_Scaled_Price']>1,1,
                                                              np.where(score_df_90['Win_Threshold_Scaled_Price']<0,0,score_df_90['Win_Threshold_Scaled_Price']))

            # inverse scaling the ANP from the pickle
            #anp_pickle=joblib.load(os.getcwd()+PICKLE+ANP_PICKLE)
            anp_pickle=joblib.load('Approved Net Price.pkl')

            # print(anp_pickle.scale_,anp_pickle.min_)
            price=anp_pickle.inverse_transform(pd.DataFrame(score_df_90['Win_Threshold_Scaled_Price']))

            score_df_90['MOP_Proba']=price
            # print(price.min(),price.max())

               #MOP is not greater than STD (list price) and not lesser than EFBC (cost price)
            #SUP is max for wholesalers, LP can be greater than SUP

            score_df_90['EFBC']= score_df_90['EFBC'].astype('float64')
            score_df_90['STD_PRICE']=pd.to_numeric(score_df_90['STD_PRICE'])
            score_df_90['MOP_Proba_Adjusted']=np.where(score_df_90['MOP_Proba']>score_df_90['STD_PRICE'],score_df_90['STD_PRICE'],score_df_90['MOP_Proba'])
            score_df_90['MOP_Proba_Adjusted']=np.where(score_df_90['MOP_Proba_Adjusted']<score_df_90['EFBC'],score_df_90['EFBC'],score_df_90['MOP_Proba_Adjusted'] )

            #Combined_output['MOP_Proba_Adjusted'] = Combined_output['MOP_Proba']
            score_df_90['Adjustment_Flag'] =np.where(score_df_90['MOP_Proba_Adjusted'] != score_df_90['MOP_Proba'],1,0)

            score_df_90.rename(columns={'MOP_Proba_Adjusted': 'Predicted_MOP_at_Target_proba', 'Win_Probability': 'Model_Predicted_Prob_at_ANP'}, inplace=True)

            #Combined_output['Win_Rate'] =Combined_output['DepVar'].sum()/Combined_output['DepVar'].count()
            # Combined_output['Win_Rate'] =Combined_output['DepVar'].mean()

            score_df_90['EFBC'] = pd.to_numeric(score_df_90['EFBC'])

            # score_df_90['ANP_Margin'] = (score_df_90['Approved Net Price']- score_df_90['EFBC'])/score_df_90['Approved Net Price']
            # score_df_90['MOP_Margin'] = (score_df_90['Predicted_MOP_at_Target_proba']- score_df_90['EFBC'])/score_df_90['Predicted_MOP_at_Target_proba']
            # print(score_df_90['Adjustment_Flag'].mean())

            score_df_90['sup_lagged_median'] = score_df_90['sup_lagged_median'].astype('float64')
            score_df_90['Approved Net Price'] = score_df_90['Approved Net Price'].astype('float64')

            score_df_90['% Diff RP & SUP'] = (score_df_90['Approved Net Price'] - score_df_90['sup_lagged_median'])/score_df_90['Approved Net Price']
            score_df_90['Final MOP'] = np.where(score_df_90['% Diff RP & SUP']>0,score_df_90['sup_lagged_median'],
                np.where(score_df_90['% Diff RP & SUP']>-0.01,score_df_90['sup_lagged_median'],
                    np.where((score_df_90['% Diff RP & SUP']<-0.01) & (score_df_90['MOP_Proba']<score_df_90['sup_lagged_median']),score_df_90['MOP_Proba'],
                        np.where((score_df_90['% Diff RP & SUP']<-0.01) & (score_df_90['MOP_Proba']>score_df_90['sup_lagged_median']),score_df_90['sup_lagged_median'],score_df_90['Approved Net Price']))))

            score_df_90['Unadjusted_MOP']= score_df_90['Final MOP']

            score_df_90['Final MOP']=np.where(score_df_90['Final MOP']>score_df_90['sup_lagged_median'],score_df_90['sup_lagged_median'],score_df_90['Final MOP'])
            score_df_90['MOP_SUP_flg']= np.where(score_df_90['Unadjusted_MOP']==score_df_90['Final MOP'],0,1)

        #    print('Win_Probability' in score_df_90.columns)
            # score_df_90['Final MOP_Margin'] = (score_df_90['Final MOP']- score_df_90['EFBC'])/score_df_90['Final MOP']

        else:
        #     print('Warning: No RFQs for MOP model scoring')
            for x in ['Final MOP','Unadjusted_MOP','MOP_SUP_flg','% Diff RP & SUP','MOP_Proba_Adjusted','MOP_Proba']:
                score_df_90[x]=0

        #         continue

        final_df= score_df_90

        del score_df_90

        #    
        
        return round(final_df['Final MOP'][0],2)

In [6]:
pp = IPEpricePrediction()

In [7]:
pp.predict_priceSP(1, 1, 300)

242.1

In [86]:
for i in range(10,15) :
    qty = 100
    sku = 9
    SP = pp.predict_priceSP(i, sku, qty)
    MOP = pp.predict_priceMOP(i, sku, qty, SP)
    
    display(pp.ipe_master_.loc[(pp.ipe_master_['Reseller'] == i) & (pp.ipe_master_['SKU'] == sku)])
    print(SP, ",", MOP)

[0.02090788]
0    0.020908
Name: Win_Probability, dtype: float64


,SKU,Reseller,FINAL_GROSS_MARGIN,Final_Qty_var,Sub_Market_Segment_val,discount_last_10_transactions,sup_lagged_median,sup_lagged_median_,SUP,DOA,Weeks_on_hand_inventory,Video_seg,Quote_Recency_Flag,Plan_TAM_ratio_Flag,Share_Change_Overall_Flag,itd_net_pos_revenue_Flag,STD_PRICE,Target_winrate,EFBC
280,9,10,0.559706,1,190.5294,0.344392,240.029927,184.638405,520,480.0,200,0,1,0,0,0,590.0,0.491176,184.4114


228.64 , 228.64
[0.02169158]
0    0.021692
Name: Win_Probability, dtype: float64


,SKU,Reseller,FINAL_GROSS_MARGIN,Final_Qty_var,Sub_Market_Segment_val,discount_last_10_transactions,sup_lagged_median,sup_lagged_median_,SUP,DOA,Weeks_on_hand_inventory,Video_seg,Quote_Recency_Flag,Plan_TAM_ratio_Flag,Share_Change_Overall_Flag,itd_net_pos_revenue_Flag,STD_PRICE,Target_winrate,EFBC
281,9,11,0.559706,23,190.5294,0.076585,198.486286,152.681758,520,480.0,200,0,1,0,0,0,590.0,0.491176,184.4114


197.13 , 198.49
[0.02302719]
0    0.023027
Name: Win_Probability, dtype: float64


...

172.49 , 172.49
[0.02238434]
0    0.022384
Name: Win_Probability, dtype: float64


,SKU,Reseller,FINAL_GROSS_MARGIN,Final_Qty_var,Sub_Market_Segment_val,discount_last_10_transactions,sup_lagged_median,sup_lagged_median_,SUP,DOA,Weeks_on_hand_inventory,Video_seg,Quote_Recency_Flag,Plan_TAM_ratio_Flag,Share_Change_Overall_Flag,itd_net_pos_revenue_Flag,STD_PRICE,Target_winrate,EFBC
283,9,13,0.544211,56,256.8101,0.153907,184.638405,142.029542,590,450.0,200,0,1,0,0,0,590.0,0.491176,184.4114


180.92 , 180.92
[0.02220305]
0    0.022203
Name: Win_Probability, dtype: float64


,SKU,Reseller,FINAL_GROSS_MARGIN,Final_Qty_var,Sub_Market_Segment_val,discount_last_10_transactions,sup_lagged_median,sup_lagged_median_,SUP,DOA,Weeks_on_hand_inventory,Video_seg,Quote_Recency_Flag,Plan_TAM_ratio_Flag,Share_Change_Overall_Flag,itd_net_pos_revenue_Flag,STD_PRICE,Target_winrate,EFBC
284,9,14,0.504816,56,256.8101,0.043062,184.638405,142.029542,520,480.0,200,0,1,0,0,0,590.0,0.491176,184.4114


183.34 , 184.64


In [74]:
i = 13
p= pp.ipe_master_.loc[(pp.ipe_master_['Reseller'] == i) & (pp.ipe_master_['SKU'] == 9),['DOA', 'SUP', 'STD_PRICE','EFBC']]
print(p)

       DOA  SUP  STD_PRICE      EFBC
283  450.0  590      590.0  184.4114
